In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as ss
import tensorflow as tf
import joblib
import time

In [3]:
# Price for zero-coupon bond with stochastic interest rate under Vasicek's model
def ZC_Vasicek(F, r, kappa, theta, sigma, t, T):
    
    delta_T = T - t
    
    B = (1 - np.exp(-kappa * delta_T)) / kappa
    
    A = np.exp((theta - (sigma**2) / (2 * kappa**2)) * (B - delta_T) - (sigma**2 / (4 * kappa)) * B**2)
    
    bond_price = F * A * np.exp(-B * r)
    
    return bond_price

In [4]:
def rpoi(a, y, lam, t, T):
    return np.exp(lam * (T-t) * (np.exp(a) - 1) - a * y)

def rlog(a, y, mu, sig, n):
    # Calculate the exponents
    exponent1 = ((mu + a)**2 - mu**2) / (2 * sig**2)
    exponent2 = a * y / (sig**2)
    
    # Compute the final result using logarithms to avoid overflow
    log_result = (exponent1 * n) - exponent2  # Logarithmic equivalent of the division
    
    # Calculate the result using np.exp
    result = np.exp(log_result)
    
    return result

In [5]:
# MC with lognormal distribution for the size of losses
def naive_MC_log(nr, lam, D, mu, sig, T):
    h = []
    poissons = np.random.poisson(lam=lam*T, size=nr)
    for i in range(nr):
        x = np.sum(np.random.lognormal(mean=mu, sigma=sig, size = poissons[i]))
        h.append(int(x>D))
    return np.cumsum(h)/np.arange(1,nr+1)

In [6]:
# Importance sampling for default probability
def MC_IS_log_poi(nr, lam, D, mu, sig, t, T):
    # Initial variables
    a_poi = np.log(D / (lam * T * np.exp(mu+sig**2/2)))/2
    poisson_means = lam * T * np.exp(a_poi)
    a_log = np.log(D/poisson_means) - (mu+sig**2/2)
    new_mu = mu+a_log

    # Preallocate memory for cumulative sums
    h = np.zeros(nr)
    r_poi = np.zeros(nr)
    r_log = np.zeros(nr)
    
    # Loop over the number of simulations
    for i in range(nr):
        # Generate Poisson-distributed count
        poissons = np.random.poisson(lam=poisson_means)

        # Generate the exponential random variables for this count
        # Directly sum them without creating large intermediate arrays
        x = np.random.normal(loc=new_mu, scale=sig, size=poissons)
        y = np.sum(x)
        z = np.sum(np.exp(x))

        # Compute h and r for this simulation
        h[i] = int(z > D)
        r_poi[i] = rpoi(a_poi, poissons, lam, t, T)
        r_log[i] = rlog(a_log, y, mu, sig, poissons)

    # Calculate cumulative sum and return the average at each step
    cumulative_sum = np.cumsum(h * r_poi * r_log)
    cumulative_avg = cumulative_sum / np.arange(1, nr + 1)

    return cumulative_avg

In [7]:
# Zero-coupon CAT bond pricing under IS
def IS_CAT_ZC_Vasicek(F, r, kappa, theta, sigma, D, lam, mu, sig, nr, t, T):
    return ZC_Vasicek(F, r, kappa, theta, sigma, t, T)*(1-MC_IS_log_poi(nr, lam, D, mu, sig, t, T)[-1])

In [8]:
# CAT bond with coupons pricing
def CAT_C_Vasicek(D, lam, N, T):
    c_sum = 0
    dt = T/N
    
    for coupon_count in range (N):
        if lam * (coupon_count+1) * dt * np.exp(mu + sig**2 / 2) < D:
            c_sum += ZC_Vasicek(F*c, r, kappa, theta, sigma, t, T=(coupon_count+1)*dt)*(1-MC_IS_log_poi(nr, lam, D, mu, sig, t, T=(coupon_count+1)*dt)[-1])
        else:  # Case: N == 0 and lam * T * k * th > D
            c_sum += ZC_Vasicek(F*c, r, kappa, theta, sigma, t, T=(coupon_count+1)*dt)*(1-naive_MC_log(nr, lam, D, mu, sig, T=(coupon_count+1)*dt)[-1])
    
    if lam * T * np.exp(mu + sig**2 / 2) < D:
        c_sum = c_sum + ZC_Vasicek(F, r, kappa, theta, sigma, t, T)*(1-MC_IS_log_poi(nr, lam, D, mu, sig, t, T)[-1])
    else:  # Case: N == 0 and lam * T * k * th > D
        c_sum = c_sum + ZC_Vasicek(F, r, kappa, theta, sigma, t, T)*(1-naive_MC_log(nr, lam, D, mu, sig, T)[-1])
    
    return c_sum

In [9]:
F = 1  # Fixed face value
t = 0     # Fixed initial time
mu = 18.4    # Fixed lognormal shape
sig = 1    # Fixed lognormal scale
c=0.05
r=0.03
kappa=0.2
theta=0.03
sigma=0.02
nr = 10000

D=9*1000000000
lam=35
N=2
T=1

num_runs = 1000  # Number of iterations
results = []  # Store function outputs

start_time = time.time()  # Start timing

for _ in range(num_runs):
    result = CAT_C_Vasicek(D, lam, N, T)  # Run the function
    results.append(result)  # Store the result

end_time = time.time()  # End timing

# Compute statistics
total_time_MC = end_time - start_time  # Total execution time
prediction_MC = np.mean(results)  # Mean of the function outputs

# Print results
print("Prediction:", prediction_MC)
print(f"Time taken for prediction: {total_time_MC:.6f} seconds")

Prediction: 1.037670255506584
Time taken for prediction: 164.733739 seconds


In [10]:
# Define custom objects dictionary
custom_objects = {"mse": tf.keras.losses.MeanSquaredError()}

# Load the saved model
model_log = tf.keras.models.load_model("best_NN_lognormal.h5", custom_objects=custom_objects)

# Define the feature names (match the original dataset used to fit the scaler)
feature_names = ["r", "lambda", "D", "N", "T"]  # Update with actual names

# Example input: Replace this with the actual input shape expected by model
new_input = np.array([[0.03, 35, 9*1000000000, 2, 1]])  # Modify based on model's input shape

# Convert new_input to a DataFrame
new_input_df = pd.DataFrame(new_input, columns=feature_names)

# Load the fitted scaler
scaler = joblib.load("scaler_log.pkl")

# Apply the same scaling
new_input_scaled = scaler.transform(new_input_df)

new_input_scaled_batch = np.tile(new_input_scaled, (num_runs, 1))  # Duplicate input for batching
start_time = time.time()
results = model_log.predict(new_input_scaled_batch)  # Single batch prediction
end_time = time.time()

# Compute statistics
total_time_log = end_time - start_time  # Total execution time
prediction_log = np.mean(results)  # Mean of the function outputs

# Print results
print("Prediction:", prediction_log)
print(f"Time taken for prediction: {total_time_log:.6f} seconds")

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
Prediction: 1.0387813
Time taken for prediction: 0.086861 seconds


In [11]:
# Example input: Replace this with the actual input shape expected by model
new_input = np.array([[0.03, 35, 9*1000000000, 2, 1]])  # Modify based on model's input shape

# Convert new_input to a DataFrame
new_input_df = pd.DataFrame(new_input, columns=feature_names)

# Apply the same scaling
new_input_scaled = scaler.transform(new_input_df)

new_input_scaled_batch = np.tile(new_input_scaled, (num_runs, 1))  # Duplicate input for batching
start_time = time.time()
results = model_log.predict(new_input_scaled_batch)  # Single batch prediction
end_time = time.time()

# Compute statistics
total_time_log = end_time - start_time  # Total execution time
prediction_log = np.mean(results)  # Mean of the function outputs

# Print results
print("Prediction:", prediction_log)
print(f"Time taken for prediction: {total_time_log:.6f} seconds")

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 416us/step
Prediction: 1.0387813
Time taken for prediction: 0.029811 seconds


In [12]:
N=4
T=1

results = []  # Store function outputs

start_time = time.time()  # Start timing

for _ in range(num_runs):
    result = CAT_C_Vasicek(D, lam, N, T)  # Run the function
    results.append(result)  # Store the result

end_time = time.time()  # End timing

# Compute statistics
total_time_MC = end_time - start_time  # Total execution time
prediction_MC = np.mean(results)  # Mean of the function outputs

# Print results
print("Prediction:", prediction_MC)
print(f"Time taken for prediction: {total_time_MC:.6f} seconds")

Prediction: 1.1360151890941566
Time taken for prediction: 270.791549 seconds


In [13]:
# Example input: Replace this with the actual input shape expected by model
new_input = np.array([[0.03, 35, 9*1000000000, 4, 1]])  # Modify based on model's input shape

# Convert new_input to a DataFrame
new_input_df = pd.DataFrame(new_input, columns=feature_names)

# Apply the same scaling
new_input_scaled = scaler.transform(new_input_df)

new_input_scaled_batch = np.tile(new_input_scaled, (num_runs, 1))  # Duplicate input for batching
start_time = time.time()
results = model_log.predict(new_input_scaled_batch)  # Single batch prediction
end_time = time.time()

# Compute statistics
total_time_log = end_time - start_time  # Total execution time
prediction_log = np.mean(results)  # Mean of the function outputs

# Print results
print("Prediction:", prediction_log)
print(f"Time taken for one prediction: {total_time_log:.6f} seconds")

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 423us/step
Prediction: 1.1338567
Time taken for one prediction: 0.030764 seconds


In [14]:
N=8
T=2

results = []  # Store function outputs

start_time = time.time()  # Start timing

for _ in range(num_runs):
    result = CAT_C_Vasicek(D, lam, N, T)  # Run the function
    results.append(result)  # Store the result

end_time = time.time()  # End timing

# Compute statistics
total_time_MC = end_time - start_time  # Total execution time
prediction_MC = np.mean(results)  # Mean of the function outputs

# Print results
print("Prediction:", prediction_MC)
print(f"Time taken for one prediction: {total_time_MC:.6f} seconds")

Prediction: 0.4253706275342994
Time taken for one prediction: 417.915222 seconds


In [22]:
# Example input: Replace this with the actual input shape expected by model
new_input = np.array([[0.03, 35, 9*1000000000, 8, 2]])  # Modify based on model's input shape

# Convert new_input to a DataFrame
new_input_df = pd.DataFrame(new_input, columns=feature_names)

# Apply the same scaling
new_input_scaled = scaler.transform(new_input_df)

new_input_scaled_batch = np.tile(new_input_scaled, (num_runs, 1))  # Duplicate input for batching
start_time = time.time()
results = model_log.predict(new_input_scaled_batch)  # Single batch prediction
end_time = time.time()

# Compute statistics
total_time_log = end_time - start_time  # Total execution time
prediction_log = np.mean(results)  # Mean of the function outputs

# Print results
print("Prediction:", prediction_log)
print(f"Time taken for one prediction: {total_time_log:.6f} seconds")

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 373us/step
Prediction: 0.42789125
Time taken for one prediction: 0.029408 seconds


In [23]:
N=12
T=2

results = []  # Store function outputs

start_time = time.time()  # Start timing

for _ in range(num_runs):
    result = CAT_C_Vasicek(D, lam, N, T)  # Run the function
    results.append(result)  # Store the result

end_time = time.time()  # End timing

# Compute statistics
total_time_MC = end_time - start_time  # Total execution time
prediction_MC = np.mean(results)  # Mean of the function outputs

# Print results
print("Prediction:", prediction_MC)
print(f"Time taken for one prediction: {total_time_MC:.6f} seconds")

Prediction: 0.5823190587038356
Time taken for one prediction: 606.227924 seconds


In [24]:
# Example input: Replace this with the actual input shape expected by model
new_input = np.array([[0.03, 35, 9*1000000000, 12, 2]])  # Modify based on model's input shape

# Convert new_input to a DataFrame
new_input_df = pd.DataFrame(new_input, columns=feature_names)

# Apply the same scaling
new_input_scaled = scaler.transform(new_input_df)

new_input_scaled_batch = np.tile(new_input_scaled, (num_runs, 1))  # Duplicate input for batching
start_time = time.time()
results = model_log.predict(new_input_scaled_batch)  # Single batch prediction
end_time = time.time()

# Compute statistics
total_time_log = end_time - start_time  # Total execution time
prediction_log = np.mean(results)  # Mean of the function outputs

# Print results
print("Prediction:", prediction_log)
print(f"Time taken for one prediction: {total_time_log:.6f} seconds")

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 371us/step
Prediction: 0.57999146
Time taken for one prediction: 0.029497 seconds
